In [ ]:
import socket
import subprocess

In [ ]:
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

server_socket.bind(('localhost', 80))
server_socket.listen(0)
print("listening")

while  True :
    client_socket, addr = server_socket.accept()
    print("accepting")
    data = client_socket.recv(65535)    
    data = data.decode()
    print(data)
    headers = data.split("\r\n")
    filename = headers[0].split(" ")[1]
    send_header = 'HTTP/1.0 200 OK\r\n\r\n'
    
    if '.py' in filename:
        html = subprocess.check_output(['python.exe', '.'+filename ]) #실행시킬 프로그램 이름, 프로그램 파일 
        # bash 에서 `python test.py`를 실행 시키는 것과 똑같음
        print(html.decode('cp949')) #30 , string 으로 바꾼 결과, 한글이 들어간경우 `cp949`로 인코딩 해주어야 함
        client_socket.send(send_header.encode('utf-8'))
        client_socket.send(html)
        
    elif '.html' in filename :
        print("html로 빠짐")
        file = open("."+ filename, 'rt', encoding='utf-8') #텍스트로 읽어옴, w-쓰기, r-읽기, t-텍스트, b-바이너리
        html = file.read()          
        client_socket.send(send_header.encode("utf-8")) # 인코드 타입 정해주어야 클라이언트는 깨진문장으로 받지 않음
        client_socket.send(html.encode("utf-8"))
    
    else :
        send_header = 'HTTP/1.0 404 File Not Found\r\n\r\n'        
        client_socket.send(send_header.encode("utf-8"))
    client_socket.close()

In [ ]:
import socket
import threading
from datetime import datetime
import subprocess

def httpprocess(client_socket) :
    data = client_socket.recv(65535)   
    data = data.decode()
    print(data)
    try :    
        headers = data.split("\r\n")
        filename = headers[0].split(" ")[1]
        
        if '.py' in filename:
            
            html = subprocess.check_output(['python.exe', '.' + filename])
            html = html.decode('cp949')
            
            header = 'HTTP/1.0 200 OK\r\n'               
            client_socket.send(header.encode("utf-8"))
            client_socket.send("Content-Type: text/html\r\n\r\n".encode())                                    
            client_socket.send(html.encode("utf-8"))            
        elif '.html' in filename:
            file = open("."+ filename, 'rt', encoding='utf-8')
            html = file.read()    
            header = 'HTTP/1.0 200 OK\r\n'               
            client_socket.send(header.encode("utf-8"))
            client_socket.send("Content-Type: text/html\r\n\r\n".encode())            
            client_socket.send(html.encode("utf-8"))
        elif '.jpg' in filename or '.png' in filename:         
            client_socket.send('HTTP/1.1 200 OK\r\n'.encode())
            client_socket.send("Content-Type: image/jpg\r\n".encode())
            client_socket.send("Accept-Ranges: bytes\r\n\r\n".encode())
            file = open("." + filename, "rb")            
            client_socket.send(file.read())  
            file.close()               
        else :
            header = 'HTTP/1.0 404 File Not Found\r\n\r\n'        
            client_socket.send(header.encode("utf-8"))
    except Exception as e :
        print(e)         
    client_socket.close()


server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 82))
server_socket.listen(0)
print("listening")

while  True :
    client_socket, addr = server_socket.accept()
    print("accepting")
    t = threading.Thread(target=httpprocess, args=(client_socket,))
    t.start()

listening
accepting
GET /test2.py HTTP/1.1
Host: localhost:82
Connection: keep-alive
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36
Sec-Fetch-User: ?1
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9
Sec-Fetch-Site: none
Sec-Fetch-Mode: navigate
Accept-Encoding: gzip, deflate, br
Accept-Language: ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7
Cookie: username-localhost-8890="2|1:0|10:1580373995|23:username-localhost-8890|44:NTU4NDQ0YTUzNTQzNDUzZjljZTBlOTQ5NDZmYjgyNjA=|c9d61a7d1bc3e6bcbaa128aeab2efddff7fed467fbb6ac311ca5b1d12c29ded4"; username-localhost-8891="2|1:0|10:1580374324|23:username-localhost-8891|44:MmM0YzIzYTk1NTFhNDM4Y2EyY2IwZjdlNjdlYWQyNjE=|441435adc346b4a528bb36200f0ee7387740159594715080cfe9ec563bb33774"; username-localhost-8889="2|1:0|10:1580431297|23:username-localhost-8889|44:NzMzZTY5OWQyY2N

In [10]:
def render(html, data):
    sp_html = html.split(' ')
    for i,k in data.items():
        html = html.replace( '@'+i, k )
    
    return html
    
html = "hello @v1     test @v2     item @v3"
data = {'v1':"안녕하세요" , 'v2':'이순시', 'v3': '^^'}
html = render(html, data)
print(html)

hello 안녕하세요     test 이순시     item ^^
